In [1]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

import os
import json

In [2]:
path = os.getcwd()

In [3]:
newCutoff = ""
fromNotebook = True
warehouse = ""
sparkMaster = "local[*]"
retriesMax = 2

In [4]:
jars = ""

if fromNotebook:
    f = open('set_cutoff_date_bq.json',)
    config = json.load(f)
    f.close()

    newCutoff = config["cutoff_date"]
    warehouse = config["warehouse"]
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../resources/credentials/bigquery_token.json"
else:
    jars = "{0}/resources/jars/mssql-jdbc-13.2.0.jre11.jar,{0}/resources/jars/spark-bigquery-with-dependencies_2.12-0.42.2.jar".format(path)
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = f"{path}/resources/credentials/bigquery_token.json"

In [5]:
bqClient = bigquery.Client(project=warehouse["database"])
bqClient

In [ ]:
def set_cutoff_date():
    whDatabase = warehouse["database"]
    whSchema = warehouse["schema"]
    whDbTableFrom = f"`{whDatabase}`.`{whSchema}`.`LatestCutoffDate`"
    retries = 0
    isSuccessful = False

    while retries < retriesMax:    
        
        retries = retries + 1
        
        try:
    
            # check if table exists
            try:

                setSql = f"""
                    CREATE TABLE IF NOT EXISTS {whDbTableFrom} (
                        cutoff_date DATETIME
                    );
                    TRUNCATE TABLE {whDbTableFrom};
                    INSERT INTO {whDbTableFrom} (cutoff_date)
                    VALUES ('{newCutoff}');
                """

                print(setSql)
                
                insertCutoffDateJob = bqClient.query(setSql)
                
                insertCutoffDateJob.result()

                retries = retriesMax
                isSuccessful = True

            except NotFound:
                retries = retriesMax
                isSuccessful = True
            except Exception as ex:
                raise ex

        except Exception as ex:
            if retries >= retriesMax:
                raise ex

    if isSuccessful == False:
        raise Exception("Unable to set cutoff date")
    
set_cutoff_date()


                    CREATE TABLE IF NOT EXISTS `dbt-tutorial-462014`.`WideWorldImportersDW`.`LatestCutoffDate` (
                        cutoff_date DATETIME
                    );
                    INSERT INTO `dbt-tutorial-462014`.`WideWorldImportersDW`.`LatestCutoffDate` (cutoff_date)
                    VALUES ('2013-01-01 00:00:00');
                
